In [1]:
%load_ext autoreload

In [2]:
%autoreload 1

In [3]:
import biom
import torch
import torch.nn as nn
%aimport mmvec.ALR

import numpy as np

torch.manual_seed(15)

In [4]:
microbes = biom.load_table("./soil_microbes.biom")
metabolites = biom.load_table("./soil_metabolites.biom")

model = mmvec.ALR.MMvecALR(microbes, metabolites, 15, sigma_u=1, sigma_v=1)

microbes = microbes.to_dataframe().T
metabolites = metabolites.to_dataframe().T
microbes = microbes.loc[metabolites.index]

microbe_idx = microbes.columns
metabolite_idx = metabolites.columns
microbes = torch.tensor(microbes.values, dtype=torch.int)
metabolites = torch.tensor(metabolites.values, dtype=torch.int64)

microbe_relative_frequency = (microbes.T/microbes.sum(1)).T

microbe_count = microbes.shape[1]
metabolite_count = metabolites.shape[1]

In [5]:
#model = mmvec.ALR.MMvecALR(microbe_count, metabolite_count, 15, sigma_u=1, sigma_v=1)
learning_rate = 1e-3
batch_size = 200
epochs = 100
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate, maximize=True)

In [6]:
model.microbe_relative_freq

tensor([[0.2017, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0950, 0.1752, 0.0000,  ..., 0.0000, 0.0013, 0.0000],
        [0.1416, 0.0973, 0.0000,  ..., 0.0000, 0.0064, 0.0000],
        ...,
        [0.0507, 0.0000, 0.0090,  ..., 0.0058, 0.0000, 0.0051],
        [0.0382, 0.0076, 0.0025,  ..., 0.0000, 0.0009, 0.0100],
        [0.0027, 0.0135, 0.0000,  ..., 0.0000, 0.0008, 0.0036]])

In [7]:
maybe = mmvec.train.mmvec_training_loop(
                            model=model,
                            optimizer=optimizer,
                            batch_size=batch_size,
                            epochs=epochs)

loss: -14241466368.0
Batch #: 0


In [9]:
model.ranks

tensor([[ 1.1758,  0.0677, -0.5526,  ..., -0.7825, -0.5267, -0.1815],
        [ 0.2582, -0.7442,  0.0576,  ..., -1.0060, -0.2658, -0.9660],
        [ 0.9580, -0.5079, -0.8934,  ..., -0.5971, -0.0238, -0.3293],
        ...,
        [ 0.0645,  0.1041, -0.8826,  ...,  0.0947, -0.1519, -0.1774],
        [ 0.4002,  0.1209, -0.4545,  ..., -0.8281, -1.0184,  0.7384],
        [ 1.1294, -0.6548, -0.0167,  ..., -1.0560, -0.1584, -0.8694]])

In [10]:
# h = model.ranks_df - model.ranks_df.mean(axis=0)
h = model.ranks_matrix

k = model.latent_dim

AttributeError: 'MMvecALR' object has no attribute 'ranks_matrix'

In [ ]:
h.mean(dim=0).shape

In [ ]:
from torch.linalg import svd


In [ ]:
u, s, v = svd(h, full_matrices=False)

In [ ]:
u.shape, s.shape, v.shape

In [ ]:
s

In [11]:
model.U

AttributeError: 'MMvecALR' object has no attribute 'U'

In [ ]:
model.get_ordination()

In [ ]:
ranks = pd.DataFrame(model.ranks(),
                     index=microbe_idx,
                     columns=metabolite_idx)

In [ ]:
ranks

In [ ]:
for param in model.parameters():
    print(param.shape)

print(model.parameters)

In [ ]:
a = torch.randint(10, (9, 3), dtype=torch.float)
b = torch.randint(10, (3,))
a, a - a.mean(dim=0), a.mean(dim=0)

In [ ]:
bp = model.get_ordination()

In [ ]:
import os.path

In [ ]:
bp_path = os.path.join("/Users/keeganevans/Desktop/", "biplot")

In [ ]:
bp.write(bp_path)

In [ ]:
model.ranks_df